Trong hướng dẫn này, bạn sẽ học ba cách tiếp cận để **xử lý dữ liệu bị thiếu** (*dealing with missing values*). Sau đó, bạn sẽ so sánh hiệu quả của các phương pháp này trên một tập dữ liệu thực tế.

# Giới thiệu (# Introduction #)

Có nhiều lý do khiến dữ liệu có thể bị thiếu giá trị. Ví dụ:
- Một căn nhà có 2 phòng ngủ sẽ không có giá trị cho kích thước của phòng ngủ thứ ba.
- Một người tham gia khảo sát có thể chọn không tiết lộ thu nhập của họ.

Hầu hết các thư viện *machine learning* (bao gồm cả *scikit-learn*) sẽ báo lỗi nếu bạn cố gắng xây dựng một mô hình với dữ liệu có giá trị bị thiếu. Vì vậy, bạn cần chọn một trong các chiến lược xử lý dưới đây.

# Ba phương pháp tiếp cận (# Three Approaches #)

### 1) Cách đơn giản: Loại bỏ cột có giá trị bị thiếu

Cách đơn giản nhất là loại bỏ các cột có giá trị bị thiếu.  

![tut2_approach1](https://storage.googleapis.com/kaggle-media/learn/images/Sax80za.png)

Trừ khi phần lớn giá trị trong các cột bị loại bỏ đều bị thiếu, phương pháp này sẽ khiến mô hình mất đi nhiều thông tin (có thể rất hữu ích!).  

Ví dụ cực đoan, hãy tưởng tượng một tập dữ liệu có 10.000 hàng, trong đó một cột quan trọng chỉ bị thiếu duy nhất một giá trị. Phương pháp này sẽ loại bỏ toàn bộ cột đó!

### 2) Một phương án tốt hơn: Điền giá trị bị thiếu (*Imputation*)

**Imputation** (*điền giá trị thay thế*) lấp đầy các giá trị bị thiếu bằng một số nào đó. Ví dụ, chúng ta có thể điền vào giá trị trung bình của từng cột.  

![tut2_approach2](https://storage.googleapis.com/kaggle-media/learn/images/4BpnlPA.png)

Giá trị được điền vào có thể không hoàn toàn chính xác trong hầu hết các trường hợp, nhưng thường dẫn đến mô hình chính xác hơn so với việc loại bỏ toàn bộ cột.

### 3) Mở rộng phương pháp Imputation

*Imputation* là phương pháp phổ biến và thường mang lại hiệu quả tốt. Tuy nhiên, giá trị được điền vào có thể có xu hướng cao hơn hoặc thấp hơn giá trị thực tế (mà dữ liệu gốc không thu thập được). Ngoài ra, các hàng có giá trị bị thiếu có thể có đặc điểm riêng biệt theo một cách nào đó.  

Trong trường hợp này, mô hình có thể đưa ra dự đoán tốt hơn nếu biết được những giá trị nào đã bị thiếu ban đầu.  

![tut3_approach3](https://storage.googleapis.com/kaggle-media/learn/images/UWOyg4a.png)

Phương pháp này vẫn thực hiện *imputation* để điền giá trị như trước. Đồng thời, với mỗi cột có giá trị bị thiếu trong tập dữ liệu ban đầu, chúng ta thêm một cột mới để đánh dấu vị trí của các giá trị đã bị thay thế.  

Trong một số trường hợp, điều này giúp cải thiện kết quả đáng kể. Tuy nhiên, trong những trường hợp khác, nó không mang lại lợi ích gì.

# Ví dụ (# Example #)

Trong ví dụ này, chúng ta sẽ làm việc với tập dữ liệu [Melbourne Housing](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home).  

Mô hình của chúng ta sẽ sử dụng thông tin như số lượng phòng và diện tích đất để dự đoán giá nhà.

Chúng ta sẽ không tập trung vào bước tải dữ liệu. Thay vào đó, bạn có thể hình dung rằng chúng ta đã có sẵn dữ liệu huấn luyện và kiểm định trong các biến:  
- `X_train`, `X_valid` (tập đặc trưng huấn luyện và kiểm định)  
- `y_train`, `y_valid` (tập mục tiêu huấn luyện và kiểm định)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

### Xác định hàm đánh giá chất lượng của từng phương pháp (# Define Function to Measure Quality of Each Approach #)

Chúng ta sẽ định nghĩa một hàm `score_dataset()` để so sánh các phương pháp khác nhau trong việc xử lý dữ liệu bị thiếu.  

Hàm này sẽ báo cáo *mean absolute error* ([MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) - sai số tuyệt đối trung bình) từ một mô hình *random forest*.

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Điểm số từ phương pháp 1 (Loại bỏ cột có giá trị bị thiếu) (# Score from Approach 1 (Drop Columns with Missing Values) #)

Vì chúng ta đang làm việc với cả tập huấn luyện (*training set*) và tập kiểm định (*validation set*), cần đảm bảo rằng các cột bị loại bỏ trong cả hai *DataFrame* đều giống nhau.

In [11]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


### Điểm số từ phương pháp 2 (Điền giá trị bị thiếu - Imputation) (# Score from Approach 2 (Imputation) #)

Tiếp theo, chúng ta sử dụng [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) để thay thế các giá trị bị thiếu bằng giá trị trung bình của từng cột.

Mặc dù là một phương pháp đơn giản, việc điền giá trị trung bình thường mang lại hiệu quả khá tốt (tuy nhiên, điều này còn phụ thuộc vào từng tập dữ liệu).  

Mặc dù các nhà thống kê đã thử nghiệm nhiều phương pháp phức tạp hơn để xác định giá trị thay thế (chẳng hạn như **regression imputation** - *điền giá trị bằng hồi quy*), nhưng trên thực tế, các chiến lược phức tạp này thường không mang lại lợi ích đáng kể khi sử dụng với các mô hình *machine learning* tiên tiến.

In [12]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


Chúng ta thấy rằng **Phương pháp 2** có MAE thấp hơn so với **Phương pháp 1**, nghĩa là **Phương pháp 2** hoạt động tốt hơn trên tập dữ liệu này.

### Điểm số từ phương pháp 3 (Mở rộng phương pháp Imputation) (# Score from Approach 3 (An Extension to Imputation) #)

Tiếp theo, chúng ta thực hiện *imputation* để điền giá trị bị thiếu, đồng thời tạo một cột đánh dấu vị trí các giá trị đã được thay thế.

In [13]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


Như chúng ta có thể thấy, **Phương pháp 3** hoạt động kém hơn một chút so với **Phương pháp 2**.

### Vậy tại sao phương pháp imputation lại tốt hơn so với việc loại bỏ cột? (# So, why did imputation perform better than dropping the columns? #)

Tập dữ liệu huấn luyện có 10.864 hàng và 12 cột, trong đó có ba cột chứa giá trị bị thiếu. Đối với mỗi cột, số lượng giá trị bị thiếu chiếm ít hơn một nửa tổng số hàng.  

Do đó, nếu loại bỏ các cột này, chúng ta sẽ mất đi rất nhiều thông tin hữu ích. Vì vậy, không có gì ngạc nhiên khi *imputation* mang lại hiệu suất tốt hơn.

In [14]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


# Kết luận (# Conclusion #)

Như thường thấy, việc điền giá trị bị thiếu (*imputation*) (trong **Phương pháp 2** và **Phương pháp 3**) mang lại kết quả tốt hơn so với khi đơn giản loại bỏ các cột có giá trị bị thiếu (trong **Phương pháp 1**).

# Đến lượt bạn! (# Your Turn #)

Hãy tự mình so sánh các phương pháp xử lý giá trị bị thiếu trong **[bài tập này](https://www.kaggle.com/kernels/fork/3370280)**!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*